<b> Dataset Link : </b> https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud

<b> Problem Statement </b> 

It is important that credit card companies are able to recognize fraudulent credit card transactions so that customers are not charged for items that they did not purchase.

<b> About the dataset </b> 

The dataset contains transactions made by credit cards in September 2013 by European cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-sensitive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

In [1]:
## importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv("creditcard.csv")

In [9]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


* Amount is given in USD
* PCA is implemented on all the features and converted it into numeric data as the information is confidential.
* Class 0 says non-fraud transaction whereas class 1 says fraud transaction.

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

* No missing values. 

In [11]:
## Checking the distribution of class feature
data['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

* Out of the complete dataset, 284315 are non-fraudlent transactions whereas 492 are fraud transactions.<b> A very imbalanced dataset. </b>

In [12]:
## splitting the class feature into legit and fraudulent transactions
legit = data[data['Class']==0]
fraudulent = data[data['Class']==1]

In [13]:
legit['Amount'].describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [14]:
fraudulent['Amount'].describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [15]:
data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


<b> DEALING WITH IMBALANCED DATASET </b>

<b> Under Sampling- Creating a sample having similar distribution as the original dataset. </b>

In [16]:
legit_sample = legit.sample(n=492)
# concatenating two dataframes
sampled_dataset = pd.concat([legit_sample, fraudulent], axis=0) 

In [17]:
sampled_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
142674,84872.0,-0.707673,0.454520,1.996292,0.192702,-0.239913,0.095053,0.116467,0.278719,0.358502,...,0.168385,0.746715,-0.115809,0.139195,-0.179589,0.484787,0.366111,0.196149,29.99,0
191009,129071.0,1.534727,-1.167011,-1.154164,0.027534,0.376491,1.860681,-0.527059,0.563712,0.909286,...,-0.242910,-0.938251,0.346491,-0.968297,-0.845225,0.208149,-0.040564,-0.036227,192.40,0
99941,67350.0,-1.052014,1.034444,-0.573200,-2.108434,0.183927,-0.569420,0.279580,0.693161,0.217451,...,0.219707,0.523437,-0.286867,-0.790323,0.046715,-0.794383,-0.174111,0.008722,19.07,0
101836,67957.0,-1.101238,1.298832,1.377953,-0.381607,0.113303,-0.137106,0.658107,0.068789,0.168837,...,-0.301023,-0.466812,0.001362,-0.031987,-0.190265,0.071311,0.421254,0.084505,8.90,0
137222,82070.0,1.121773,-0.123515,0.019074,0.024665,-0.412840,-0.984146,0.257428,-0.145133,-0.222216,...,-0.358571,-1.318699,0.143618,0.332972,0.019575,0.599759,-0.123015,0.004187,77.40,0


In [18]:
sampled_dataset['Class'].value_counts()

0    492
1    492
Name: Class, dtype: int64

<b> Now we have a balanced dataset having 492 fraudulent transactions and 492 legit transactions </b>

In [19]:
sampled_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,101444.528455,0.079321,0.043320,-0.132562,-0.097385,-0.046692,0.004545,-0.008465,-0.004682,-0.023025,...,-0.028446,-0.008369,0.013103,0.008112,0.001447,-0.027728,-0.018591,-0.002941,-0.005745,80.510041
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


<b> The difference in mean from the sampled dataset and the original dataset is similar. This tells us that the nature of the sampled dataset hasn't changed and we have got a good sample. If this is not the case, then we can say that we got a bad sample. </b>

<b> SPLITTING THE DATA INTO INPUT AND OUTPUT </b>

In [20]:
X=sampled_dataset.drop(columns=['Class'],axis=1)
y=sampled_dataset['Class']

<b> SPLITTING DATA INTO TRAINING AND TESTING </b>

In [21]:
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2, stratify=y, random_state=2)

#stratify helps to evenly distribute y data in both x-test and y-test. 

In [22]:
print(X.shape, X_train.shape, X_test.shape) 

(984, 30) (787, 30) (197, 30)


* We have successfully split the data into train and test.

<b> MODEL TRAINING </b>

<b> Since it is a binary classification problem, I am going to implement Logistic Regression </b>

In [32]:
model= LogisticRegression()

In [33]:
model.fit(X_train, y_train)

LogisticRegression()

In [34]:
X_train_prediction=model.predict(X_train)
training_data_accuracy= accuracy_score(X_train_prediction,y_train)

In [35]:
print("Accuracy on training data: ", training_data_accuracy)

Accuracy on training data:  0.9466327827191868


<b> But how does the model perform on unseen data ? </b>

In [36]:
X_test_prediction=model.predict(X_test)
test_data_accuracy= accuracy_score(X_test_prediction,y_test)

In [37]:
print("Accuracy on testing data: ", test_data_accuracy)

Accuracy on testing data:  0.949238578680203


<b> Training data accuracy and Testing data accuracy are similar so our model is not overfitted </b>